In [ ]:
#Best params from prophet II.ipynb

import os
import json

best_params = {
    'window_size': 186,
    'iqr_multiplier': 2.8736874205941256,
    'hurricane_window': 4,
    'changepoint_prior_scale': 0.0783519917195005,
    'seasonality_prior_scale': 2.99223202049866,
    'holidays_prior_scale': 10.495427649405375,
    'seasonality_mode': 'additive',
    'yearly_seasonality': 7
}

output_dir = 'output'

with open(os.path.join(output_dir, 'best_params.json'), 'w') as f:
    json.dump(best_params, f)

In [ ]:
# Cleaned and processed data from prophet II.ipynb

